In [2]:
import sweetviz as sv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

c:\Users\Pc\anaconda3\envs\JBapp\Lib\site-packages\sweetviz\graph.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [3]:
df = pd.read_csv('da_internship_task_dataset.csv')
df = pd.DataFrame(df)

In [3]:
#basic info
print(df.info(), "\n")
print(df.describe(), "\n")
print(df.isnull().sum(), "\n")
print(df.head(), "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122746 entries, 0 to 122745
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   uuid          122746 non-null  object 
 1   day_id        122746 non-null  object 
 2   license       122746 non-null  object 
 3   model         122746 non-null  object 
 4   feature       122746 non-null  object 
 5   requests_cnt  122746 non-null  float64
 6   spent_amount  122746 non-null  float64
dtypes: float64(2), object(5)
memory usage: 6.6+ MB
None 

        requests_cnt   spent_amount
count  122746.000000  122746.000000
mean       51.260742      12.227798
std       167.977380      37.588494
min         1.000000       0.270000
25%        18.000000       5.140000
50%        32.000000       8.020000
75%        54.000000      13.060000
max     12900.000000    2599.000000 

uuid            0
day_id          0
license         0
model           0
feature         0
requests_cnt    0
s

In [6]:
#sweetviz report
report = sv.analyze(df)
report.show_html('Sweetviz_Report_Pairwise.html')

                                             |          | [  0%]   00:00 -> (? left)

Report Sweetviz_Report_Pairwise.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
#Sweetviz report without pairwise analysis
report = sv.analyze(df, pairwise_analysis='off')

report.show_html('Sweetviz_Report_No_Pairwise.html')

                                             |          | [  0%]   00:00 -> (? left)

Report Sweetviz_Report_No_Pairwise.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
